In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Load the training and test datasets
training_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
test_passenger_ids = test_df['PassengerId']

In [3]:
#check for the missing data

from pandas.io.formats.info import DataFrameInfo

temp = pd.DataFrame(training_df)

info = DataFrameInfo(data=temp)
infodf = pd.DataFrame(
  {'Column': info.ids, 
   'Non-Null Count':info.non_null_counts, 
   'Dtype':info.dtypes})

pd.set_option('display.max_rows', None)
print(infodf.sort_values(by=['Non-Null Count']))

                  Column  Non-Null Count    Dtype
Cabin              Cabin             204   object
Age                  Age             714  float64
Embarked        Embarked             889   object
PassengerId  PassengerId             891    int64
Survived        Survived             891    int64
Pclass            Pclass             891    int64
Name                Name             891   object
Sex                  Sex             891   object
SibSp              SibSp             891    int64
Parch              Parch             891    int64
Ticket            Ticket             891   object
Fare                Fare             891  float64


In [4]:
training_targets = training_df['Survived']
training_df.drop(['Cabin','PassengerId','Name','Survived','Ticket'],axis=1,inplace=True)
Age_mean = training_df['Age'].mean()
training_df['Age'].fillna(Age_mean, inplace=True)
Embarked_mode = training_df['Embarked'].mode()

training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [5]:
one_hot_columns_list = ['Embarked','Pclass','Sex']
one_hot_encoded_data = pd.get_dummies(training_df[one_hot_columns_list],columns = one_hot_columns_list, drop_first=True).astype(int)
one_hot_encoded_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Embarked_Q  891 non-null    int64
 1   Embarked_S  891 non-null    int64
 2   Pclass_2    891 non-null    int64
 3   Pclass_3    891 non-null    int64
 4   Sex_male    891 non-null    int64
dtypes: int64(5)
memory usage: 34.9 KB


In [6]:
numeric_continuous_columns_list=['Age','Parch','SibSp','Fare']
for column in numeric_continuous_columns_list:
    training_df[column] = (training_df[column] - training_df[column].min()) / (training_df[column].max() - training_df[column].min())
training_df.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.367921,0.065376,0.063599,0.062858
std,0.836071,0.163383,0.137843,0.134343,0.096995
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.271174,0.000000,0.000000,0.015440
50%,3.000000,0.367921,0.000000,0.000000,0.028213
75%,3.000000,0.434531,0.125000,0.000000,0.060508
max,3.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
temp = pd.DataFrame(test_df)

info = DataFrameInfo(data=temp)
infodf = pd.DataFrame(
  {'Column': info.ids, 
   'Non-Null Count':info.non_null_counts, 
   'Dtype':info.dtypes})

pd.set_option('display.max_rows', None)
print(infodf.sort_values(by=['Non-Null Count']))

                  Column  Non-Null Count    Dtype
Cabin              Cabin              91   object
Age                  Age             332  float64
Fare                Fare             417  float64
PassengerId  PassengerId             418    int64
Pclass            Pclass             418    int64
Name                Name             418   object
Sex                  Sex             418   object
SibSp              SibSp             418    int64
Parch              Parch             418    int64
Ticket            Ticket             418   object
Embarked        Embarked             418   object


In [8]:
test_df.drop(['Cabin','PassengerId','Name','Ticket'],axis=1,inplace=True)
Age_mean = test_df['Age'].mean()
test_df['Age'].fillna(Age_mean, inplace=True)
Embarked_mode = test_df['Embarked'].mode()

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


In [9]:
one_hot_encoded_data_test = pd.get_dummies(test_df[one_hot_columns_list],columns = one_hot_columns_list, drop_first=True).astype(int)
one_hot_encoded_data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Embarked_Q  418 non-null    int64
 1   Embarked_S  418 non-null    int64
 2   Pclass_2    418 non-null    int64
 3   Pclass_3    418 non-null    int64
 4   Sex_male    418 non-null    int64
dtypes: int64(5)
memory usage: 16.5 KB


In [10]:
for column in numeric_continuous_columns_list:
    test_df[column] = (test_df[column] - test_df[column].min()) / (test_df[column].max() - test_df[column].min())
test_df.describe()

,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,417.000000
mean,2.265550,0.396975,0.055921,0.043594,0.069540
std,0.841838,0.166617,0.112095,0.109048,0.109124
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.301068,0.000000,0.000000,0.015412
50%,3.000000,0.396975,0.000000,0.000000,0.028213
75%,3.000000,0.469207,0.125000,0.000000,0.061484
max,3.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
preprocessed_training_df = pd.concat([training_df[numeric_continuous_columns_list], one_hot_encoded_data], axis=1, join='inner')
preprocessed_test_df = pd.concat([test_df[numeric_continuous_columns_list], one_hot_encoded_data_test], axis=1, join='inner')

In [12]:
preprocessed_training_df.describe()

,Age,Parch,SibSp,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Sex_male
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.367921,0.063599,0.065376,0.062858,0.086420,0.722783,0.206510,0.551066,0.647587
std,0.163383,0.134343,0.137843,0.096995,0.281141,0.447876,0.405028,0.497665,0.477990
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.271174,0.000000,0.000000,0.015440,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.367921,0.000000,0.000000,0.028213,0.000000,1.000000,0.000000,1.000000,1.000000
75%,0.434531,0.000000,0.125000,0.060508,0.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
preprocessed_training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         891 non-null    float64
 1   Parch       891 non-null    float64
 2   SibSp       891 non-null    float64
 3   Fare        891 non-null    float64
 4   Embarked_Q  891 non-null    int64  
 5   Embarked_S  891 non-null    int64  
 6   Pclass_2    891 non-null    int64  
 7   Pclass_3    891 non-null    int64  
 8   Sex_male    891 non-null    int64  
dtypes: float64(4), int64(5)
memory usage: 62.8 KB


In [14]:
preprocessed_test_df.describe()

,Age,Parch,SibSp,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Sex_male
count,418.000000,418.000000,418.000000,417.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,0.396975,0.043594,0.055921,0.069540,0.110048,0.645933,0.222488,0.521531,0.636364
std,0.166617,0.109048,0.112095,0.109124,0.313324,0.478803,0.416416,0.500135,0.481622
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.301068,0.000000,0.000000,0.015412,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.396975,0.000000,0.000000,0.028213,0.000000,1.000000,0.000000,1.000000,1.000000
75%,0.469207,0.000000,0.125000,0.061484,0.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
preprocessed_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         418 non-null    float64
 1   Parch       418 non-null    float64
 2   SibSp       418 non-null    float64
 3   Fare        417 non-null    float64
 4   Embarked_Q  418 non-null    int64  
 5   Embarked_S  418 non-null    int64  
 6   Pclass_2    418 non-null    int64  
 7   Pclass_3    418 non-null    int64  
 8   Sex_male    418 non-null    int64  
dtypes: float64(4), int64(5)
memory usage: 29.5 KB


In [16]:
preprocessed_training_df[preprocessed_training_df.isna().any(axis=1)]

,Age,Parch,SibSp,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Sex_male


In [17]:
preprocessed_test_df[preprocessed_test_df.isna().any(axis=1)]

,Age,Parch,SibSp,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Sex_male
152,0.795595,0.0,0.0,NaN,0,1,0,1,1


In [18]:
Fare_mean = training_df['Fare'].mean()
preprocessed_test_df['Fare'].fillna(Fare_mean, inplace=True)

In [19]:
preprocessed_test_df[preprocessed_test_df.isna().any(axis=1)]

,Age,Parch,SibSp,Fare,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Sex_male


In [20]:
# https://scikit-learn.org/stable/auto_examples/linear_model/plot_iris_logistic.html
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5, multi_class='multinomial')

# Create an instance of Softmax and fit the data.
logreg.fit(preprocessed_training_df, training_targets)



LogisticRegression(C=100000.0, multi_class='multinomial')

In [21]:
#performance of training
y_train_pred = logreg.predict(preprocessed_training_df)
print(y_train_pred)

[0 1 1 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0
 0 1 1 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0
 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 0 1
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1
 0 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1
 0 1 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 0 0
 0 1 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 1 1 1
 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 0
 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 1
 0 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0
 0 0 0 1 0 1 0 0 0 0 0 0 

In [22]:
# errors
error_train = training_targets - y_train_pred
error_train_squared = np.power(error_train,2)
print("Training error mean: ", error_train.mean())
print("Training squared error mean: ", error_train_squared.mean())

Training error mean:  0.03367003367003367
Training squared error mean:  0.19528619528619529


In [23]:
error_train_sum = (error_train.abs()).sum()
print("Training absolute error sum, percentage error: ", error_train_sum, (error_train_sum/891)*100 )

Training absolute error sum, percentage error:  174 19.52861952861953


In [24]:
y_test_pred = logreg.predict(preprocessed_test_df)

In [25]:
print(y_test_pred)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1
 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [26]:
y_test_pred_df = pd.DataFrame(y_test_pred, columns = ['Survived'])

In [27]:
submission_list = pd.concat([test_passenger_ids, y_test_pred_df], axis=1, join='inner')


In [28]:
submission_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [29]:
print(submission_list)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
5            897         0
6            898         1
7            899         0
8            900         1
9            901         0
10           902         0
11           903         0
12           904         1
13           905         0
14           906         1
15           907         1
16           908         0
17           909         0
18           910         1
19           911         1
20           912         0
21           913         0
22           914         1
23           915         1
24           916         1
25           917         0
26           918         1
27           919         0
28           920         0
29           921         0
30           922         0
31           923         0
32           924         0
33           925         0
34           926         0
35           927         0
3

In [30]:
submission_list.to_csv('/kaggle/working/predictions_for_submission', index=False)
